# Challenge State of Data

### Sobre os dados

O questionário foi dividido em 8 partes, e dentro de cada uma das partes temos as perguntas e opções de escolha.

- Parte 1 - Dados demográficos
- Parte 2 - Dados sobre carreira
- Parte 3 - Desafios dos gestores de times de dados
- Parte 4 - Conhecimentos na área de dados
- Parte 5 - Objetivos na área de dados
- Parte 6 - Conhecimentos em Engenharia de Dados/DE
- Parte 7 - Conhecimentos em Análise de Dados/DA
- Parte 8 - Conhecimentos em Ciências de Dados/DS

Cada pergunta é dividida em Parte, Letra da Pergunta, Número da Opção escolhida
- Exemplo: P3a_1 = Parte 3, pergunta (a), opção (1)

## 0. Configuração do ambiente

### 0.1. Bibliotecas

In [1]:
import pandas as pd
import numpy as np

### 0.2. Dados

In [3]:
data = pd.read_csv('State_of_data_2022.csv')

## 1. Visão geral do dataframe

In [4]:
# número de linhas e de colunas

data.shape

(4271, 353)

In [36]:
# como esta base tem muitas colunas, vou checar os temas pelas partes

### Parte 1 - Dados demográficos

In [30]:
parte1 = data.filter(regex = "^\('P1", axis=1).columns.to_frame(name = 'lista').reset_index(drop = True)

separar = parte1['lista'].str.split(',', n = 1, expand = True)

parte1['codigo'] = separar[0]
parte1['codigo'] = parte1['codigo'].str.replace('(', '')
parte1['codigo'] = parte1['codigo'].str.replace("'", '')

parte1['descricao'] = separar[1]
parte1['descricao'] = parte1['descricao'].str.replace(')', '')
parte1['descricao'] = parte1['descricao'].str.replace("'", '')

parte1.drop(['lista'], axis = 1)

,codigo,descricao
0,P1_a,Idade
1,P1_a_1,Faixa idade
2,P1_b,Genero
3,P1_c,Cor/raca/etnia
4,P1_d,PCD
5,P1_e,experiencia_profissional_prejudicada
6,P1_f,aspectos_prejudicados
7,P1_g,vive_no_brasil
8,P1_i,Estado onde mora
9,P1_i_1,uf onde mora


### Parte 2 - Dados sobre carreira

In [29]:
parte2 = data.filter(regex = "^\('P2", axis=1).columns.to_frame(name = 'lista').reset_index(drop = True)

separar = parte2['lista'].str.split(',', n = 1, expand = True)

parte2['codigo'] = separar[0]
parte2['codigo'] = parte2['codigo'].str.replace('(', '')
parte2['codigo'] = parte2['codigo'].str.replace("'", '')

parte2['descricao'] = separar[1]
parte2['descricao'] = parte2['descricao'].str.replace(')', '')
parte2['descricao'] = parte2['descricao'].str.replace("'", '')

parte2.drop(['lista'], axis = 1)

,codigo,descricao
0,P2_a,Qual sua situação atual de trabalho?
1,P2_b,Setor
2,P2_c,Numero de Funcionarios
3,P2_d,Gestor?
4,P2_e,Cargo como Gestor
5,P2_f,Cargo Atual
6,P2_g,Nivel
7,P2_h,Faixa salarial
8,P2_i,Quanto tempo de experiência na área de dados ...
9,P2_j,Quanto tempo de experiência na área de TI/Eng...


### Parte 3 - Desafios dos gestores de times de dados

In [31]:
parte3 = data.filter(regex = "^\('P3", axis=1).columns.to_frame(name = 'lista').reset_index(drop = True)

separar = parte3['lista'].str.split(',', n = 1, expand = True)

parte3['codigo'] = separar[0]
parte3['codigo'] = parte3['codigo'].str.replace('(', '')
parte3['codigo'] = parte3['codigo'].str.replace("'", '')

parte3['descricao'] = separar[1]
parte3['descricao'] = parte3['descricao'].str.replace(')', '')
parte3['descricao'] = parte3['descricao'].str.replace("'", '')

parte3.drop(['lista'], axis = 1)

,codigo,descricao
0,P3_a,Qual o número aproximado de pessoas que atuam...
1,P3_b,Quais desses papéis/cargos fazem parte do tim...
2,P3_b_1,Analytics Engineer
3,P3_b_2,Engenharia de Dados/Data Engineer
4,P3_b_3,Analista de Dados/Data Analyst
5,P3_b_4,Cientista de Dados/Data Scientist
6,P3_b_5,Database Administrator/DBA
7,P3_b_6,Analista de Business Intelligence/BI
8,P3_b_7,Arquiteto de Dados/Data Architect
9,P3_b_8,Data Product Manager/DPM


### Parte 4 - Conhecimentos na área de dados

In [32]:
parte4 = data.filter(regex = "^\('P4", axis=1).columns.to_frame(name = 'lista').reset_index(drop = True)

separar = parte4['lista'].str.split(',', n = 1, expand = True)

parte4['codigo'] = separar[0]
parte4['codigo'] = parte4['codigo'].str.replace('(', '')
parte4['codigo'] = parte4['codigo'].str.replace("'", '')

parte4['descricao'] = separar[1]
parte4['descricao'] = parte4['descricao'].str.replace(')', '')
parte4['descricao'] = parte4['descricao'].str.replace("'", '')

parte4.drop(['lista'], axis = 1)

,codigo,descricao
0,P4_a,"Mesmo que esse não seja seu cargo formal, voc..."
1,P4_a_1,Atuacao
2,P4_b,Quais das fontes de dados listadas você já an...
3,P4_b_1,Dados relacionais (estruturados em bancos SQL
4,P4_b_2,Dados armazenados em bancos NoSQL
...,...,...
95,P4_i_19,Grafana
96,P4_i_20,TIBCO Spotfire
97,P4_i_21,Pentaho
98,P4_i_22,Fazemos todas as análises utilizando apenas E...


### Parte 5 - Objetivos na área de dados

In [33]:
parte5 = data.filter(regex = "^\('P5", axis=1).columns.to_frame(name = 'lista').reset_index(drop = True)

separar = parte5['lista'].str.split(',', n = 1, expand = True)

parte5['codigo'] = separar[0]
parte5['codigo'] = parte5['codigo'].str.replace('(', '')
parte5['codigo'] = parte5['codigo'].str.replace("'", '')

parte5['descricao'] = separar[1]
parte5['descricao'] = parte5['descricao'].str.replace(')', '')
parte5['descricao'] = parte5['descricao'].str.replace("'", '')

parte5.drop(['lista'], axis = 1)

,codigo,descricao
0,P5_a,Qual seu objetivo na área de dados?
1,P5_b,Qual oportunidade você está buscando?
2,P5_c,Há quanto tempo você busca uma oportunidade n...
3,P5_d,Como tem sido a busca por um emprego na área ...


In [35]:
data.filter(regex = "^\('P5", axis=1).sort_values(by = "('P5_a ', 'Qual seu objetivo na área de dados?')").head()

,"('P5_a ', 'Qual seu objetivo na área de dados?')","('P5_b ', 'Qual oportunidade você está buscando?')","('P5_c ', 'Há quanto tempo você busca uma oportunidade na área de dados?')","('P5_d ', 'Como tem sido a busca por um emprego na área de dados?')"
808,Acabei de migrar para área de dados e sigo me ...,NaN,NaN,NaN
3086,"Agregar mais conhecimento sobre dados, para po...",NaN,NaN,NaN
1520,Apenas conhecimentos: Busco conhecimentos em d...,NaN,NaN,NaN
1195,Apenas conhecimentos: Busco conhecimentos em d...,NaN,NaN,NaN
1194,Apenas conhecimentos: Busco conhecimentos em d...,NaN,NaN,NaN


### Parte 6 - Conhecimentos em engenharia de dados

In [37]:
parte6 = data.filter(regex = "^\('P6", axis=1).columns.to_frame(name = 'lista').reset_index(drop = True)

separar = parte6['lista'].str.split(',', n = 1, expand = True)

parte6['codigo'] = separar[0]
parte6['codigo'] = parte6['codigo'].str.replace('(', '')
parte6['codigo'] = parte6['codigo'].str.replace("'", '')

parte6['descricao'] = separar[1]
parte6['descricao'] = parte6['descricao'].str.replace(')', '')
parte6['descricao'] = parte6['descricao'].str.replace("'", '')

parte6.drop(['lista'], axis = 1)

,codigo,descricao
0,P6_a,Quais das opções abaixo fazem parte da sua ro...
1,P6_a_1,Desenvolvo pipelines de dados utilizando ling...
2,P6_a_2,Realizo construções de ETLs em ferramentas co...
3,P6_a_3,Crio consultas através da linguagem SQL para ...
4,P6_a_4,Atuo na integração de diferentes fontes de da...
5,P6_a_5,"Modelo soluções de arquitetura de dados, cria..."
6,P6_a_6,Desenvolvo/cuido da manutenção de repositório...
7,P6_a_7,"Atuo na modelagem dos dados, com o objetivo d..."
8,P6_a_8,"Cuido da qualidade dos dados, metadados e dic..."
9,P6_a_9,Nenhuma das opções listadas refletem meu dia ...


### Parte 7 - Conhecimentos em análise de dados

In [38]:
parte7 = data.filter(regex = "^\('P7", axis=1).columns.to_frame(name = 'lista').reset_index(drop = True)

separar = parte7['lista'].str.split(',', n = 1, expand = True)

parte7['codigo'] = separar[0]
parte7['codigo'] = parte7['codigo'].str.replace('(', '')
parte7['codigo'] = parte7['codigo'].str.replace("'", '')

parte7['descricao'] = separar[1]
parte7['descricao'] = parte7['descricao'].str.replace(')', '')
parte7['descricao'] = parte7['descricao'].str.replace("'", '')

parte7.drop(['lista'], axis = 1)

,codigo,descricao
0,P7_1,Quais das opções abaixo fazem parte da sua ro...
1,P7_a_1,Processo e analiso dados utilizando linguagen...
2,P7_a_2,Realizo construções de dashboards em ferramen...
3,P7_a_3,Crio consultas através da linguagem SQL para ...
4,P7_a_4,Utilizo APIs para extrair dados e complementa...
5,P7_a_5,Realizo experimentos e estudos utilizando met...
6,P7_a_6,Desenvolvo/cuido da manutenção de ETLs utiliz...
7,P7_a_7,"Atuo na modelagem dos dados, com o objetivo d..."
8,P7_a_8,Desenvolvo/cuido da manutenção de planilhas p...
9,P7_a_9,Utilizo ferramentas avançadas de estatística ...


### Parte 8 - Conhecimentos em ciências de dados

In [39]:
parte8 = data.filter(regex = "^\('P8", axis=1).columns.to_frame(name = 'lista').reset_index(drop = True)

separar = parte8['lista'].str.split(',', n = 1, expand = True)

parte8['codigo'] = separar[0]
parte8['codigo'] = parte8['codigo'].str.replace('(', '')
parte8['codigo'] = parte8['codigo'].str.replace("'", '')

parte8['descricao'] = separar[1]
parte8['descricao'] = parte8['descricao'].str.replace(')', '')
parte8['descricao'] = parte8['descricao'].str.replace("'", '')

parte8.drop(['lista'], axis = 1)

,codigo,descricao
0,P8_a,Quais das opções abaixo fazem parte da sua ro...
1,P8_a_1,Estudos Ad-hoc com o objetivo de confirmar hi...
2,P8_a_2,Sou responsável pela coleta e limpeza dos dad...
3,P8_a_3,Sou responsável por entrar em contato com os ...
4,P8_a_4,Desenvolvo modelos de Machine Learning com o ...
5,P8_a_5,Sou responsável por colocar modelos em produç...
6,P8_a_6,Cuido da manutenção de modelos de Machine Lea...
7,P8_a_7,Realizo construções de dashboards em ferramen...
8,P8_a_8,Utilizo ferramentas avançadas de estatística ...
9,P8_a_9,"Crio e dou manutenção em ETLs, DAGs e automaç..."


## 1. Sanity check

## 2. Análise exploratória